In [1]:
pip install pandas-datareader

     |████████████████████████████████| 107 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.5 MB 12.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install plotly

     |████████████████████████████████| 13.2 MB 4.4 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=0a933cad30cead21d5d195d09e69a94d4a977be041db2ea710b62a008a23cbfc
  Stored in directory: /home/jovyan/.cache/pip/wheels/ac/cb/8a/b27bf6323e2f4c462dcbf77d70b7c5e7868a7fbe12871770cf
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pandas_datareader import data
from datetime import datetime
from datetime import date
from sympy.solvers import solve
from sympy import Symbol
import pandas as pd
import plotly.express as px

#### Actively-managed mutual funds (or exchange-trade funds (ETF)) charge a fee for administrative, management, advertising and other expenses related to the fund. The fee is charged as a percentage of the asset's value on a daily basis. For example, if a mutual fund has an expense ratio of $1.39\%$ and the invested amount is $\$ 50,000$, then over the course of one year, the total amount deducted from the investment will be $\$ 375$. $50000\ast (0.75/100)=375$. The investor will not see the charges in the statements he receives, because the investment company handles this type of operations in-house. So, if the fund has an annual return of $0\%$ for the year, the investor will gradually see his initial  $\$ 50,000$ gradually decreases to  $\$ 49,625$ $50000-375=49625$. Hence, for actively-managed EFT's, the fund's annual return MUST be higher than the fund's expense ratio for the investor not to lose money. 

#### I have designed the following program to (1) calculate the required annual yield to achieve an investor's target value in one yeare based on the information entered by the investor; (2) calculate, based on the fund's year-to-date return, what kind of performance the fund needs to deliver for the rest of the year for the investor to achieve the required annual yield.

## Calculate the required annual yield to achieve year end target value of investment.

#### Enter the fund's expense ratio, without the percentage sign, then hit Enter on the keyboard.

In [4]:
expense_ratio=input()

1.39


#### Enter the value of the account at the beginning of the year, then hit Enter.

In [5]:
initial_value=input()

10000


#### Enter the target value of the account in one year, then hit Enter.

In [6]:
target_value=input()

12000


In [7]:
def required_yield():
    x=Symbol("x")
    annual_yield=solve(float(initial_value)*(1+x)*(1-float(expense_ratio)/100) - float(target_value), x)
    annual_yield=round(float(annual_yield[0])*100,2)
    return print("Based on the information you enter, you need an annual yield of ",annual_yield,"% to achieve the target value of your investment in one year.")

In [8]:
required_yield()

Based on the information you enter, you need an annual yield of  21.69 % to achieve the target value of your investment in one year.


## Assess the fund's performance before year end.

#### Enter your required annual yield, without the percentage sign, the hit Enter on the keyboard.

In [9]:
required_yield=input()

21.69


#### Enter the fund's year-to-date yield, without the percentage sign, then hit Enter on the keyboard.

In [10]:
ytd_yield=input()

3.34


In [11]:
def still_need():
    x=Symbol("x")
    still_need=solve((1+float(ytd_yield)/100)*(1+x)-1-float(required_yield)/100,x)
    still_need=round(still_need[0]*100,2)
    return print("Based on the fund's year_to_date return so far, the fund needs to return ",still_need,"% for the rest of the year for you to achieve your required annual yield.")

In [12]:
still_need()

Based on the fund's year_to_date return so far, the fund needs to return  17.76 % for the rest of the year for you to achieve your required annual yield.


#### I just ran the program on a hypothetic $\$10,000$ investment in *Fidelity Advisor Focused Emerging Markets Fund Class A (FAMKX)*. The fund has an expense ratio of $1.39\%$. If the investor wants to have $\$12,000$ in his account in one year, the fund needs to deliver an annual return of $21.69\%$. Otherwise, the investor will not achieve his goal. As of today, April 4, 2021, the Fidelity fund's year-to-date return is $3.34\%$. For the rest of the year, the fund needs to deliver an additional return of $17.76\%$ for the investor to realize his target of having $\$12,000$ in his account by year end.

## The following program will help the investor keep track his his account value and shows how far his account is from his target value at year end. As the program above, this program can be used to keep track of any actively managed ETF, not just the Fidelity fund in the example above. The investor just needs to enter the fund's ticker symbol, the program will run the rest.

#### Enter the fund's ticker symbol, then hit Enter on the keyboard.

In [13]:
ticker=input()

FAMKX


#### Enter the account's value at the beginning of the year, then hit Enter on the keyboard.

In [14]:
initial_value2=input()

10000


In [21]:
famkx = data.DataReader(f'{ticker}', 'yahoo', '2004-03-29')
famkx=famkx.reset_index()

famkx["Daily_change"]=famkx["Close"].pct_change()
famkx=famkx.dropna()

famkx['Year'] = pd.DatetimeIndex(famkx['Date']).year
current=date.today().year
famkx1=famkx[famkx["Year"]==current]
famkx1=famkx1.drop(["Year"],axis=1).reset_index(drop=True)

expense_rate=float(expense_ratio)/100
nav=pd.DataFrame((10000*famkx1["Daily_change"])*(1-expense_rate)).cumsum().reset_index(drop=True)+10000
nav=pd.concat([famkx1[["Date"]],nav],axis=1)
nav.columns=["Date","Net Asset Value"]

fig = px.line(nav,x="Date", y="Net Asset Value")
fig.update_layout(title="Year-to-date performance of a hypothetical $10,000 investment in Fidelity Advisor Focused Emerging Markets Fund Class A (FAMKX) ",
                 width=1000)
fig.show()